In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Load the dataset
data = pd.read_csv('assets/Logistic_Regression.csv', on_bad_lines='skip')



In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 669638 entries, 0 to 669638
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   password  669638 non-null  object
 1   strength  669638 non-null  int64 
 2   features  669638 non-null  object
dtypes: int64(1), object(2)
memory usage: 20.4+ MB


In [ ]:

# Convert string labels to numeric
le = LabelEncoder()
data['strength'] = le.fit_transform(data['strength'])

# Drop rows with missing values if any
data.dropna(inplace=True)

In [ ]:
import re

def extract_features(password):
    features = []
    features.append(len(password))  # Length of the password
    features.append(sum(c.isupper() for c in password))  # Number of uppercase letters
    features.append(sum(c.islower() for c in password))  # Number of lowercase letters
    features.append(sum(c.isdigit() for c in password))  # Number of digits
    features.append(len(re.findall(r'\W', password)))  # Number of special characters
    return features

# Apply feature extraction to all passwords
data['features'] = data['password'].apply(extract_features)


In [ ]:
data.head()

,password,strength,features
0,kzde5577,1,"[8, 0, 4, 4, 0]"
1,kino3434,1,"[8, 0, 4, 4, 0]"
2,visi7k1yr,1,"[9, 0, 7, 2, 0]"
3,megzy123,1,"[8, 0, 5, 3, 0]"
4,lamborghin1,1,"[11, 0, 10, 1, 0]"


from matplotlib import pyplot as plt
import seaborn as sns
_df_0.groupby('password').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['strength']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'strength'}, axis=1)
              .sort_values('strength', ascending=True))
  xs = counted['strength']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_1.sort_values('strength', ascending=True)
for i, (series_name, series) in enumerate(df_sorted.groupby('password')):
  _plot_series(series, series_name, i)
  fig.legend(title='password', bbox_to_anchor=(1, 1), loc='upper left')
sns.despine(fig=fig, ax=ax)
plt.xlabel('strength')
_ = plt.ylabel('count()')

In [ ]:
from sklearn.model_selection import train_test_split

X = data['features'].tolist()
y = data['strength']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=1000)

In [ ]:
from sklearn.metrics import classification_report

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00     17824
           1       1.00      1.00      1.00     99360
           2       1.00      1.00      1.00     16744

    accuracy                           1.00    133928
   macro avg       1.00      1.00      1.00    133928
weighted avg       1.00      1.00      1.00    133928



In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report


def print_score(clf, X_train, y_train, X_test, y_test, train=True):
    if train:
        pred = clf.predict(X_train)
        clf_report = pd.DataFrame(classification_report(y_train, pred, output_dict=True))
        print("Train Result:\n================================================")
        print(f"Accuracy Score: {accuracy_score(y_train, pred) * 100:.2f}%")
        print("_______________________________________________")
        print(f"CLASSIFICATION REPORT:\n{clf_report}")
        print("_______________________________________________")
        print(f"Confusion Matrix: \n {confusion_matrix(y_train, pred)}\n")

    elif train==False:
        pred = clf.predict(X_test)
        clf_report = pd.DataFrame(classification_report(y_test, pred, output_dict=True))
        print("Test Result:\n================================================")
        print(f"Accuracy Score: {accuracy_score(y_test, pred) * 100:.2f}%")
        print("_______________________________________________")
        print(f"CLASSIFICATION REPORT:\n{clf_report}")
        print("_______________________________________________")
        print(f"Confusion Matrix: \n {confusion_matrix(y_test, pred)}\n")

In [ ]:
print_score(model, X_train, y_train, X_test, y_test, train=True)
print_score(model, X_train, y_train, X_test, y_test, train=False)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Train Result:
Accuracy Score: 100.00%
_______________________________________________
CLASSIFICATION REPORT:
                      0              1        2    3  accuracy      macro avg  \
precision      0.999972       1.000000      1.0  0.0  0.999996       0.749993   
recall         1.000000       0.999997      1.0  0.0  0.999996       0.749999   
f1-score       0.999986       0.999999      1.0  0.0  0.999996       0.749996   
support    71877.000000  397439.000000  66393.0  1.0  0.999996  535710.000000   

            weighted avg  
precision       0.999994  
recall          0.999996  
f1-score        0.999995  
support    535710.000000  
_______________________________________________
Confusion Matrix: 
 [[ 71877      0      0      0]
 [     1 397438      0      0]
 [     0      0  66393      0]
 [     1      0      0      0]]

Test Result:
Accuracy Score: 100.00%
_______________________________________________
CLASSIFICATION REPORT:
                 0        1        2  accuracy  